Taking User Input

In [1]:
#basic packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

# Prevent future/deprecation warnings from showing in output
import warnings
warnings.filterwarnings('ignore')


In [2]:
import nlp_utils, nltk, re, string
import contractions
import collections
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer

In [3]:
user_input = input("Type a situation to know its sentiment")
corpus = [user_input]
letters_only = re.sub(r'[^a-zA-Z]', " ", str(corpus))
letters_only=letters_only.lower()
token=nltk.sent_tokenize(letters_only)
token

['  i hate you so much that i am even disgusted when i see you']

In [4]:
## contractions are expanded and the letters are converted to lowercase.
conm = contractions.CONTRACTION_MAP
def contraction_remove(corpus_nda):
    for key,value in conm.items():
        corpus_nda = re.sub(r"{}".format(key),'{}'.format(value),corpus_nda)
        
    return corpus_nda

special = string.punctuation
def w_tokenization(corpus_nda):
    # convert into lower case
    corpus_nda = corpus_nda.lower()
    # contraction
    corpus_nda = contraction_remove(corpus_nda)
    # 
    tokens = nltk.word_tokenize(corpus_nda) # word tokens

In [5]:
### Alpha numeric characters and decimals are replaced with characters
def num_dec_al(word):
    if word.isnumeric():
        return 'xxxxxx'
    elif word.isdecimal():
        return 'xxx...'
    elif word.isalpha():
        return word
    else:
        return 'xxxaaa'

def clean_nda(token):
    tokens = nlp_utils.w_tokenization(token)
    map_list = list(map(num_dec_al,tokens))
    return " ".join(map_list)

corpus_nda = list(map(clean_nda,token))
corpus_nda

['i hate you so much that i am even disgusted when i see you']

In [6]:
df = pd.DataFrame(corpus_nda)
df= df.rename(columns={0: 'Text'})
df

,Text
0,i hate you so much that i am even disgusted wh...


In [7]:
df.replace('\d+', '', regex=True, inplace=True)
df.replace(',', '', regex=True, inplace=True)
df.replace('br', '', regex=True, inplace=True)
df.replace('"', '', regex=True, inplace=True)
df.replace("'", '', regex=True, inplace=True)
df.replace('?', '', inplace=True)
df.replace("-", '', regex=True, inplace=True)
df.replace("*", '', inplace=True)
df.replace("***", '', inplace=True)
df.replace("< />", '', regex=True, inplace=True)
df['Text'] = df['Text'].str.strip('[')
df['Text'] = df['Text'].str.strip(']')
df['Text'] = df['Text'].str.strip(')')
df['Text'] = df['Text'].str.strip('(')
df

,Text
0,i hate you so much that i am even disgusted wh...


In [8]:
df['Text'] = df['Text'].apply(nlp_utils.lemmatization_sentence)
Text=df['Text']
token = df['Text'].to_numpy()
token=nltk.sent_tokenize(str(token))
data = np.array(token)
#data
stop = stopwords.words('english')
text = data
text_tokens = word_tokenize(str(text))
tokens_without_sw = [word for word in text_tokens if not word in stop]
#print(tokens_without_sw)   


In [9]:
#data transformation - feature detection
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
tfidfFile='C:\\Users\\user\\Desktop\\SentimentAnalysisUsingNLP\\tfidf_Sentiment_Model.pk1'
tfidf = pickle.load(open(tfidfFile, "rb"))

In [10]:
X_fresh = tfidf.transform(df['Text']).toarray()
X_fresh.shape

(1, 16215)

In [11]:
#prediction using classifier
import joblib
classifier = joblib.load('LR_Classifier_Sentiment_Model')
y_pred = classifier.predict(X_fresh)
print(y_pred)

['positive']


In [12]:
import joblib
classifier = joblib.load('MNB_Classifier_Sentiment_Model')
y_pred = classifier.predict(X_fresh)
print(y_pred)

['positive']
